In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from boruta import BorutaPy
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
# from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, accuracy_score
from catboost import CatBoostRegressor
import pickle
from colorama import Fore, Back, Style
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from sklearn.linear_model import Ridge
seed = 42

from sklearn import svm
from imblearn.over_sampling import SMOTE

Matplotlib created a temporary cache directory at /tmp/matplotlib-50dcnj70 because the default path (/home/youngtaek/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:

data_path = 'PARADIGM_DB/merged_DB_at_major_revision_v4.xlsx'
merged_df = pd.read_excel(data_path, engine='openpyxl')
print (merged_df.shape)

(2324, 934)


In [7]:
train_df = merged_df[merged_df["Site_ID"].isin([3,9])]
test_df = merged_df[~merged_df["Site_ID"].isin([3,9])]

print(f"Train shape: {train_df.shape}, Unique NewID count: {train_df['NewID'].nunique()}")
print(f"Test shape: {test_df.shape}, Unique NewID count: {test_df['NewID'].nunique()}")

Train shape: (1898, 934), Unique NewID count: 443
Test shape: (426, 934), Unique NewID count: 120


In [8]:
shape_featres = [col for col in train_df.columns if 'shape' in col]
print ("number of shape features:", len(shape_featres))

# pickle 파일을 열기
with open('reproducible_features_inter_intra_analysis.pkl', 'rb') as file:
    # pickle 데이터를 읽고 역직렬화하기
    reproducible_features = pickle.load(file)

prefixed_items = ['pre_' + item for item in reproducible_features] + shape_featres

inputs = train_df.filter(regex='pre_original|pre_wavelet')
target = train_df['StatinResponder']
data = pd.concat([inputs, target], axis=1)
train_df.dropna(subset=data.columns, inplace=True)

inputs = test_df.filter(regex='pre_original|pre_wavelet')
target = test_df['StatinResponder']
data = pd.concat([inputs, target], axis=1)
test_df.dropna(subset=data.columns, inplace=True)




number of shape features: 14


/tmp/ipykernel_944588/3461878492.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset=data.columns, inplace=True)
/tmp/ipykernel_944588/3461878492.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.dropna(subset=data.columns, inplace=True)


In [10]:
target = train_df['StatinResponder']
source = train_df[prefixed_items]

# create a random forest classifier
rf = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=0)

# create the Boruta feature selector
boruta = BorutaPy(rf, n_estimators='auto', max_iter=30, verbose=0, random_state=seed)

# fit the Boruta feature selector
boruta.fit(source.values, target.values)

feature_ranking = pd.DataFrame({'feature': source.columns, 'rank': boruta.ranking_})

# Print the most important features
print(feature_ranking.sort_values('rank'))

# feature_ranking.to_excel('MajorRevision/bourta_feature_selector_feature_ranking_major_revision.xlsx', index=False)

bourtaFeatures = feature_ranking[feature_ranking['rank']==1]['feature'].values.tolist()

# Train an XGBoost model
model = XGBClassifier(objective='binary:logistic', max_depth=3, learning_rate=0.01, n_estimators=200, random_state=seed)
model.fit(source[bourtaFeatures], target)

# Get the feature importances
importances = model.feature_importances_
print (importances)

# Create a new DataFrame to store the feature importances
feature_importances = pd.DataFrame({'Feature': source[bourtaFeatures].columns, 'Importance': importances})

# Sort the DataFrame by importance in descending order
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# filter out rows with zero importance
feature_importances = feature_importances[feature_importances['Importance'] != 0]

# feature_importances.to_csv('./MajorRevision/feature_selection_StatinResponder_site39_major_revision.csv', index=False)


                                               feature  rank
348             pre_wavelet-LHH_firstorder_TotalEnergy     1
391  pre_wavelet-HLL_glrlm_LongRunHighGrayLevelEmph...     1
66              pre_wavelet-HHH_firstorder_TotalEnergy     1
392             pre_wavelet-HHL_firstorder_TotalEnergy     1
393         pre_wavelet-HHH_firstorder_RootMeanSquared     1
..                                                 ...   ...
415                 pre_original_glrlm_LongRunEmphasis   628
381  pre_original_glrlm_RunLengthNonUniformityNorma...   629
344          pre_original_gldm_SmallDependenceEmphasis   630
389                        pre_original_ngtdm_Busyness   631
601                             pre_original_glcm_Imc1   632

[697 rows x 2 columns]
[0.01412709 0.01667153 0.         0.02348031 0.0163468  0.01107448
 0.01216992 0.01311162 0.01592055 0.02113141 0.0149174  0.01640239
 0.05793812 0.0197974  0.01234543 0.01765283 0.0457972  0.
 0.01922678 0.02013523 0.03809552 0.04921979 0.0210

In [11]:
classifiers = [
    (LGBMClassifier(random_state=seed), {
        'n_estimators': [30, 40, 50, 60, 70],
        'learning_rate': [0.1, 0.5, 0.01, 0.05],
        'max_depth': [-1, 3, 5, 10, 15, 20],
        'num_leaves': [5, 10, 25, 30, 50],
        'boosting_type': ['gbdt', 'dart', 'goss'],
        'verbosity': [-1],
        'min_child_samples': [5, 10, 20, 50],
        'reg_alpha': [0, 0.01, 0.1, 1],
        'reg_lambda': [0, 0.01, 0.1, 1],
        'feature_fraction': [0.6, 0.7, 0.8, 0.9, 1.0],
        'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
    }),
    (XGBClassifier(random_state=seed), {
        'max_depth': [3, 5, 10, 15, 20, 30],
        'learning_rate': [0.1, 0.5, 0.01, 0.05],
        'n_estimators': [5, 10, 15, 20, 30, 50],
        'subsample': [0.5, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.5, 0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.3, 0.5, 1],
        'min_child_weight': [1, 3, 5, 10]
    }),
]



In [ ]:

feature_importances = pd.read_csv('./MajorRevision/feature_selection_StatinResponder_site39_secondary.csv')
print (len(feature_importances))

features = feature_importances['Feature'].values.tolist()

# # train data
y_train = train_df['StatinResponder']
X_train = train_df[features]


# # Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


y_test = test_df['StatinResponder']
X_test = test_df[features]


results = []  # 모든 iteration의 결과를 저장할 리스트

for clf, params in classifiers:
    print (clf.__class__.__name__)
    best_val_score = 0
    best_test_score = 0
    param_grid = ParameterGrid(params)
    for param in param_grid:
        clf.set_params(**param)
        clf.fit(X_train_resampled, y_train_resampled)
        train_score = roc_auc_score(y_train_resampled, clf.predict_proba(X_train_resampled)[:, 1])
        
        test_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        print (train_score, test_score)
        if test_score >= best_test_score:
            best_test_score = test_score
            results.append(
                {
                    'Classifier': clf.__class__.__name__,
                    'Params': param,
                    'Train_score': train_score,
                    'Test_score': test_score
                })
            
            dump(clf, './MajorRevision/{}_segment_prediction_StatinResponder_site39_secondary_{}.joblib'.format(clf.__class__.__name__, exp_name))
            print ("train score", train_score, "test score", test_score)
            # print (clf.__class__.__name__, "model save", './MajorRevision/{}_segment_prediction.joblib'.format(clf.__class__.__name__), "score", best_score)


df_results = pd.DataFrame(results)
df_results.to_csv("./MajorRevision/{}_segment_prediction_StatinResponder_site39_secondary_{}.csv".format(clf.__class__.__name__, exp_name), index=False)
print("All experiment results saved to CSV.")